# VICReg 

VICReg (Variance-Invariance-Covariance regularization) is a self-supervised model for visual representation learning  without annotations.
It learns by creating two augmented views of the same image using agressive data augmentations (mostly crop as in SimCLR) and then
aligning views. However, it does not use contrastive learning to avoid representation collapse but rather 2 key principles:

1. Variance maximization of each feature in the latent space (maximize expressivity)
2. Covariance minimization across features of the same image (minimize redundancy)

## Key components

- **Data augmentations**: VicReg uses cropping with resizing, gaussian blur, gaussian noise and flip to create diverse views
- **Backbone**: 3D-CNN such as ResNet are usually employed to deal with 3D volumes such as MRI
- **Projection head**: a small MLP is used to map features from CNN to a latent space where VicReg loss is applied. The output dimension is usually much larger than the input dimension (typically 8192 vs 2048), thus it is called "expander".
- **VICReg loss**:  this loss encourages similar pairs to align (invariance), variance of each embedding feature to be maximized (variance) and covariance across features to be minimized (covariance)

**Reference**: [VICReg: Variance-Invariance-Covariance Regularization for Self-Supervised Learning](https://arxiv.org/abs/2105.04906)


In [ ]:
# This example requires the following dependencies to be installed:
# pip install -e NeuroCLAV

from neuroclav.data.datasets.openbhb import OpenBHB
from neuroclav.manifold.vicreg import VICReg
from neuroclav.transforms import SimCLRTransform
from neuroclav.transforms import VolumeTransform


dataset = OpenBHB("/neurospin/hc/openBHB", target="age", modality="vbm", split="train")

vicreg = VICReg(
    encoder="resnet18_3d",
    n_components=16, 
    augmentation=SimCLRTransform(input_size=(1, 128, 128, 128)),
    test_transform=VolumeTransform(input_size=(1, 128, 128, 128)),
    max_iteration=100,
    batch_size=128,
    num_workers=10
)

vicreg.fit(dataset)
